# Developer Handbook - Plotting.py
- module contains functions to create plots according to data and choosen GUI options
- list of functions:
    - basic plot functions: creates plottype, functionname represents plottype (e.g. plot_violin(), plot_box(), ..)
    - style_plot(): functions adds additional plotfeatures to basic plot (e.g. axislabels, suptitle, legend)
    - overlay_data_point_and_error_bars(): adds standard deviation, mean and option datapoint to plot
- parameter glossar:
    - data : DataFrame containing the data to plot.
    - treatment_to_compare: The column to use for grouping data
    - y_column: The column containing the dependent variable
 
## Packages 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## Function - basic plot functions  
### plot_violin()
- parameter: data, treatment_to_compare (determined by GUI/user), y_column ("values")
- violin plot from seaborn package, only outer line of distribution is shown 
- additional withing function: overlay_data[...]() and style_plot()

In [ ]:
def plot_violin(data, treatment_to_compare, y_column):
    sns.violinplot(x=treatment_to_compare, y='Value', data=data, inner=None)
    overlay_data_points_and_error_bars(data, treatment_to_compare, "Violin Plot")
    style_plot("Violin Plot", treatment_to_compare, y_column)

### plot_box()
- parameter: data, treatment_to_compare (determined by GUI/user), y_column ("values")
- box-plot from seaborn package
- additional within function: overlay_data[...]() and style_plot()

In [ ]:
def plot_box(data, treatment_to_compare, y_column):
    sns.boxplot(x=treatment_to_compare, y='Value', data=data)
    overlay_data_points_and_error_bars(data, treatment_to_compare, "Box Plot")
    style_plot("Box Plot", treatment_to_compare, y_column)

### plot_bar()
- parameter: data, treatment_to_compare (determined by GUI/user), y_column ("values")
- bar-plot from seaborn package, estimator is mean (default option) and no ci shown 
- additional within function: overlay_data[...]() and style_plot()

In [ ]:
def plot_bar(data, treatment_to_compare, y_column):        
    sns.barplot(x=treatment_to_compare, y='Value', data=data, estimator='mean', ci=None)
    overlay_data_points_and_error_bars(data, treatment_to_compare, "Bar Plot")
    style_plot("Bar Plot", treatment_to_compare, y_column)

### plot_line()
- parameter: data, treatment_to_compare (determined by GUI/user)
- data-values grouped by treatment to compare, mean and standard deviation computed for each treatment to compare group 
- line-plot from seaborn package
- additional within function: overlay_data[...]() and style_plot()

In [ ]:
def plot_line(data, treatment_to_compare, y_column):
    summary_stats = data.groupby(treatment_to_compare)['Value'].agg(['mean', 'std']).reset_index()
    sns.lineplot(data=data, x=treatment_to_compare, y='Value', estimator='mean', ci=None)
    plt.errorbar(summary_stats[treatment_to_compare], summary_stats['mean'], yerr=summary_stats['std'],
                     fmt='o', color='k', capsize=5, label='Mean ± SD')
    style_plot("Line Plot", treatment_to_compare, y_column)

### plot_hist()
- parameter: data, treatment_to_compare (determined by GUI/user), y_column ("values")
- histogramm-plot from seaborn package

In [ ]:
def plot_hist(data, treatment_to_compare, y_column):
    sns.histplot(data['Value'], bins=20)
    style_plot("Histogram", treatment_to_compare, y_column)

### plot_heatmap()
- parameter: data, treatment_to_compare (determined by GUI/user)
- check if treatment to compare is in "Treatment1" or "Treatment2", else: value error raised
- create unique group based on treatment_to_compare
- pivot tabel to aggregate data: aggregation according to unique_treatment and line-ID ("Accession"), mean ist used to aggregate
- plot heatmap from aggregated data, colormap: `YlGnBu`, annotations deactivated 
- heatmap-plot from seaborn package
- additional within function: style_plot()

In [ ]:
def plot_heatmap(data, treatment_to_compare):
    if treatment_to_compare not in ["Treatment1", "Treatment2"]:
        raise ValueError("Unsupported treatment specified.")

    if treatment_to_compare == "Treatment1":
        unique_treatment = data['Treatment1']
    elif treatment_to_compare == "Treatment2":
        unique_treatment = data['Treatment1'] + "_" + data['Treatment2']
    else:
        unique_treatment = data['Treatment1'] + "_" + data['Treatment2'] + "_" + data['Sample']

    data['Unique_Treatment'] = unique_treatment

    heatmap_data = pd.pivot_table(data, index="Accession", columns="Unique_Treatment", values="Value", aggfunc="mean")
    sns.heatmap(heatmap_data, cmap="YlGnBu", annot=False)
    style_plot("Heatmap", treatment_to_compare)

### plot_swarm()
- parameter: data, treatment_to_compare (determined by GUI/user), y_column ("values")
- swarm-plot from seaborn package
- additional within function: overlay_data[...]() and style_plot()

In [ ]:
def plot_swarm(data, treatment_to_compare, y_column):
    sns.swarmplot(x=treatment_to_compare, y='Value', data=data)
    overlay_data_points_and_error_bars(data, treatment_to_compare, "Swarm Plot")
    style_plot("Swarm Plot", treatment_to_compare, y_column)

## Function - style_plot()
- paramter: plot_type (handed over by function call), treatment_to_compare, y-column optional (handed over by function call)
- created elements:
    - suptitle with plot type- and treatment to compare information, default size for fontsize= 18 and position set via ha="center" and x=0.5
    - x-axis label: treatment to compare information
    - when an y-axis ist given by function call, y-axis label contains "value" information
    - x-tick labels are rotated by 45° clockwise
    - automated optimised plot display (avoided overlappping)
    - a legend is shown

In [ ]:
def style_plot(plot_type, treatment_to_compare, y_column=None): 
    plt.suptitle(f'{plot_type} for {treatment_to_compare}', fontsize = 18, ha='center', x=0.5)
    plt.xlabel(f'{treatment_to_compare}')
    if y_column: 
        plt.ylabel('Values')
    plt.xticks(rotation=45)
    plt.tight_layout() 
    plt.legend()

## Function - overlay_data_points_and_error_bars()
- paramter: data, treatment to compare, plottype (handed over by function call)
-  check if plot type is not "Box Plot"
    - if true:
        - data grouped by treatment to compare, from value column mean and standard deviation computed for each group
        - mean and error bar added per group to plot, data from summary stats used, display elements set
- if plot type is "Violin Plot", data points are added via sns.stripplot function to the display (red points, semitransparent and jittered) 

In [ ]:
def overlay_data_points_and_error_bars(data, treatment_to_compare, plot_type):
    if plot_type != "Box Plot":
        summary_stats = data.groupby(treatment_to_compare)['Value'].agg(['mean', 'std', 'count']).reset_index()
        plt.errorbar(summary_stats[treatment_to_compare], summary_stats['mean'], yerr=summary_stats['std'],
                     fmt='o', color='k', capsize=5, label='Mean ± SD')
    
    if plot_type in ("Violin Plot", "Box Plot", "Box Plot"):
        sns.stripplot(x=treatment_to_compare, y='Value', data=data, color='red', alpha=0.6, jitter=True)
       